In [ ]:
import pandas as pd
import math

# Read file with latin1 encoding
with open("../data/raw/cleveland.data", "r", encoding="latin1") as f:
    lines = f.readlines()

data_rows = []

for line in lines:
    parts = line.strip().split()
    
    # Skip empty lines or lines containing 'name'
    if not parts or "name" in parts:
        continue

    clean_row = []
    for x in parts:
        # Remove null bytes and other non-digit characters
        x_clean = x.replace("\x00", "").strip()
        
        # Convert -9 -> NaN
        if x_clean == "-9" or x_clean == "":
            clean_row.append(float("nan"))
        else:
            try:
                clean_row.append(float(x_clean))
            except ValueError:
                # If still cannot convert, append NaN
                clean_row.append(float("nan"))
    
    data_rows.append(clean_row)

# Convert to DataFrame
df = pd.DataFrame(data_rows)

print(df)

# Save cleaned CSV
df.to_csv("../data/processed/cleveland_clean.csv", index=False)




      0     1      2     3      4     5     6     7   8   9   ...  18  19  20  \
0    1.0   0.0   63.0   1.0    NaN   NaN   NaN   NaN NaN NaN  ... NaN NaN NaN   
1    NaN   1.0  145.0   1.0  233.0   NaN  50.0  20.0 NaN NaN  ... NaN NaN NaN   
2    1.0   NaN    1.0   2.0    2.0   3.0  81.0   0.0 NaN NaN  ... NaN NaN NaN   
3    0.0   0.0    0.0   0.0    1.0  10.5   6.0  13.0 NaN NaN  ... NaN NaN NaN   
4  150.0  60.0  190.0  90.0  145.0  85.0   0.0   0.0 NaN NaN  ... NaN NaN NaN   

   21  22  23  24  25  26  27  
0 NaN NaN NaN NaN NaN NaN NaN  
1 NaN NaN NaN NaN NaN NaN NaN  
2 NaN NaN NaN NaN NaN NaN NaN  
3 NaN NaN NaN NaN NaN NaN NaN  
4 NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 28 columns]
(2689, 28)
